In [30]:
import pandas as pd
import numpy as np
from datetime import datetime, date
#import category_encoders as ce
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.impute import SimpleImputer
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import RandomizedSearchCV
import math
from sklearn.dummy import DummyClassifier
from sklearn.metrics import classification_report

In [31]:
#Calculates current age of fight at present.
def calculate_age(born):
    today = datetime.now()
    return today.year - born.year - ((today.month, today.day) < (born.month, born.day))
def age_at_fight(born, date):
    return (date - born).astype('<m8[Y]')
def remove_other_side(col):
    return col[:-11] if col.endswith('_other_side') else col
def find_other_fighter(row):
    # Split the matchup into two competitors
    fighter = row['BOUT'].split(' vs. ')
    # Return the competitor that is not equal to the given competitor
    return fighter[1] if row['FIGHTER'] == fighter[0] else fighter[0]

# Function to determine the outcome for each competitor
def find_outcomes(row):
    competitors = row['BOUT'].split(' vs. ')
    outcome = row['OUTCOME']
    
    # Determine outcomes based on the matchup and outcome
    if outcome == 'W/L':
        competitor_outcome = 'W' if row['FIGHTER'] == competitors[0] else 'L'
        competitor_two_outcome = 'L' if row['FIGHTER'] == competitors[0] else 'W'
    else:  # outcome == 'L/W'
        competitor_outcome = 'L' if row['FIGHTER'] == competitors[0] else 'W'
        competitor_two_outcome = 'W' if row['FIGHTER'] == competitors[0] else 'L'
    
    return pd.Series([competitor_outcome, competitor_two_outcome])

    #Apply the function to create the new columns

def current_stats(dataframe):
    # Drop specified columns
    dataframe = dataframe.drop(columns=['EVENT', 'BOUT', 'OUTCOME', 'WEIGHT_opponent', 'REACH_opponent', 'STANCE_opponent', 
                                        'METHOD', 'ROUND', 'TIME FORMAT', 'FIGHTER2', 'fighter_outcome', 
                                        'fighter_outcome_2', 'LOCATION'])
    # Drop rows where 'DATE' is NaN
    dataframe = dataframe.dropna(subset=['DATE'])

    # Group by 'FIGHTER' and find the index of the most recent 'DATE'
    idx = dataframe.groupby('FIGHTER')['DATE'].idxmax()
    idx = idx.dropna()  # Drop any NaN values from the index

    # Select the rows with the most recent stats for each fighter
    fighter_current = dataframe.loc[idx]
    return fighter_current

def compare_fighters(fighter1, fighter2):
    fighter_a = new_stats[1].loc[new_stats[1]['FIGHTER']==fighter1]
    fighter_b = new_stats[1].loc[new_stats[1]['FIGHTER']==fighter2]
    comparative_stats = pd.concat([fighter_a, fighter_b])
    return comparative_stats
    

In [37]:
def fight_stats(stat_path, event_path, outcome_path, fighter_path):
    data = pd.read_csv(stat_path)
    print(f'{len(data)-len(data.dropna())} columns were dropped')
    
    #Drop the null rows
    data = data.dropna()
    #Drop the percentage columns, we can calculate these later
    data = data.drop(columns=['SIG.STR. %', 'TD %', 'ROUND'])
    to_split = ['SIG.STR.', 'TOTAL STR.', 'TD', 'HEAD', 'BODY', 'LEG', 'DISTANCE', 'CLINCH', 'GROUND'] 
    to_split_rename = ['sig_strike', 'total_strike', 'take_down', 'head_strike', 'body_strike', 'leg_strike', 'distance_strike', 'clinch_strike', 'ground_strike']
    
    #split and rename columns with 'landed of attempted' form
    for i in range(len(to_split)):
        data[[f'{to_split_rename[i]}_landed', f'{to_split_rename[i]}_attempted']] = data.pop(to_split[i]).str.split(' of ', expand=True)
    
    #replace the blank ctrl time with 0:0 and split for minutes and second
    data['CTRL'] = data['CTRL'].replace('--', '0:0')
    data[['min_ctrl', 's_ctrl']] = data.pop('CTRL').str.split(':', expand=True)
    
    #convert str splits into integers
    data = data.astype('int', errors='ignore')
    
    #find total ctrl time in seconds
    data['total_ctrl_s'] = (data['min_ctrl'] * 60) + data['s_ctrl']
    data = data.drop(columns=['min_ctrl', 's_ctrl'])
    #sum the stats by fighter and bout so that the rounds are gone and we just have stats per fight
    data = data.groupby(['EVENT', 'BOUT', 'FIGHTER']).sum().reset_index()
    data = data.reset_index()

    #wrangle fight outcome data 
    outcome = pd.read_csv(outcome_path)
    outcome['TIME'] = outcome['TIME'].astype(str)
    outcome[['DURATION_MIN', 'DURATION_SEC']] = outcome['TIME'].str.split(':', expand=True)
    outcome['DURATION_MIN'] = outcome['DURATION_MIN'].astype(float) 
    outcome['DURATION_SEC'] = outcome['DURATION_SEC'].astype(float)
    outcome['TIME'] = (outcome['DURATION_MIN'] * 60 ) + outcome['DURATION_SEC']
    outcome['TIME'] = ((outcome['ROUND'].astype(float) - 1) * 5 * 60) + outcome['TIME']
    outcome = outcome.drop(columns=['DETAILS', 'URL', 'REFEREE', 'DURATION_MIN', 'DURATION_SEC'])
    outcome = outcome.apply(lambda x: x.str.strip() if x.dtype == 'object' else x)
    outcome['BOUT'] = outcome['BOUT'].str.replace(r'\s+', ' ', regex=True)
    data = pd.merge(data, outcome, on=['EVENT', 'BOUT'], how='left')
    data = data.drop_duplicates(subset=['EVENT', 'BOUT', 'FIGHTER'], keep='first')
    data['FIGHTER2'] = data.apply(find_other_fighter, axis=1)
    data[['fighter_outcome', 'fighter_outcome_2']] = data.apply(find_outcomes, axis=1)
    
    #make opponent dataframe for defensive stats
    o_data = data[['EVENT', 'BOUT', 'FIGHTER', 'FIGHTER2', 'KD', 'SUB.ATT', 'REV.', 'sig_strike_landed', 'sig_strike_attempted', 'total_strike_landed', 'total_strike_attempted', 'take_down_landed', 'take_down_attempted', 'head_strike_landed', 'head_strike_attempted', 'body_strike_landed', 'body_strike_attempted', 'leg_strike_landed', 'leg_strike_attempted', 'distance_strike_landed', 'distance_strike_attempted', 'clinch_strike_landed', 'clinch_strike_attempted', 'ground_strike_landed', 'ground_strike_attempted', 'total_ctrl_s']]
    
    o_data = o_data.rename(columns={'FIGHTER': 'FIGHTER2', 'FIGHTER2': 'FIGHTER'})
    
    
    
    
    data = pd.merge(data, o_data, left_on=['EVENT', 'BOUT', 'FIGHTER', 'FIGHTER2'], right_on=['EVENT', 'BOUT', 'FIGHTER', 'FIGHTER2'], suffixes=('_1', '_by_opponent'))
    
    
    #wrangle events data:
    events = pd.read_csv(event_path)
    events['DATE'] = pd.to_datetime(events['DATE'], format='%B %d, %Y')
    events[['CITY', 'STATE', 'COUNTRY']] = events['LOCATION'].str.split(',', expand=True)
    events = events.drop(columns=['STATE', 'COUNTRY', 'CITY', 'URL'])
    events = events.apply(lambda x: x.str.strip() if x.dtype == 'object' else x)
    events = events[(events['DATE'] >= '2000-01-01')]
    data = pd.merge(data, events, on=['EVENT'], how='left')

    
    # Calculate the cumulative sum of stats for each person
    #(we don't want round to be summed, it is informative the way it is)
    data['ROUND'] = data['ROUND'].astype('str')
    map_wl = {'W': 1,
           'L': 0}
    data['loss_count'] = data['fighter_outcome_2'].map(map_wl)
    data['win_count'] = data['fighter_outcome'].map(map_wl)
    

    data = data.sort_values(['FIGHTER', 'DATE'], ascending=[True, True])
    accumulated_columns = []
    columns_to_accumulate = []
    for column in list(data.columns):
        if data[column].dtype == 'int':
            columns_to_accumulate.append(column)
            accumulated_columns.append(f'{column}_total')
        elif data[column].dtype == 'float':
            columns_to_accumulate.append(column)
            accumulated_columns.append(f'{column}_total')
        else:
            continue
    
    rolling_last_two = [f'{thing}_last_two' for thing in columns_to_accumulate]
    rolling_last_three = [f'{thing}_last_three' for thing in columns_to_accumulate]
    data[rolling_last_two] = data.groupby('FIGHTER')[columns_to_accumulate].rolling(window=2).sum().reset_index(level=0, drop=True)
    data[rolling_last_three] = data.groupby('FIGHTER')[columns_to_accumulate].rolling(window=3).sum().reset_index(level=0, drop=True)
    data[accumulated_columns] = data.groupby('FIGHTER')[columns_to_accumulate].cumsum()

    #wrangling individual fighter data
    fighters = pd.read_csv(fighter_path)
    fighters['STANCE'] = fighters['STANCE'].fillna('Orthodox')
    fighters['STANCE'] = fighters['STANCE'].replace('--', 'Orthodox')
    fighters = fighters[~fighters.apply(lambda x: x.str.contains("--", na=False)).any(axis=1)]
    fighters['DOB'] = pd.to_datetime(fighters['DOB'], format="%b %d, %Y")
    fighters[['height_feet', 'height_inches']] = fighters.pop('HEIGHT').str.split("' ", expand=True)
    fighters['height_inches'] = fighters['height_inches'].str.replace('"', '')
    fighters['HEIGHT'] = (fighters['height_feet'].astype('int') *12) + fighters['height_inches'].astype('int')
    fighters['WEIGHT'] = fighters['WEIGHT'].str.replace(' lbs.', '')
    fighters['REACH'] = fighters['REACH'].str.replace('"', '')
    fighters = fighters.drop(columns=['height_inches', 'height_feet', 'URL'])
    fighters = fighters.drop_duplicates(subset=['FIGHTER'])
    opponents = fighters.rename(columns={'FIGHTER':'FIGHTER2'})
    data = pd.merge(data, fighters, on='FIGHTER', how='left')
    data = pd.merge(data, opponents, on='FIGHTER2', how='left', suffixes=('_1', '_opponent'))
    data['age_1'] = ((((data['DATE'] - data['DOB_1']).astype('<m8[s]')).astype(int)) / 31556952).astype(int)
    
    #to gather stats for proportions
    landed = []
    attempted = []
    for thing in to_split_rename:
        for col in list(data.columns):
            if f'{thing}_landed' in col:
                landed.append(col)
            elif f'{thing}_attempted' in col:
                attempted.append(col)
            else:
                continue
                
    for thing in to_split_rename:
        for i in range(len(attempted)):
            if thing in attempted[i]:
                if 'opponent' in attempted[i]:
                    if '_last_three' in attempted[i]:
                        data[f'{thing}_opponent_last_three_proportion'] = data[landed[i]] / data[attempted[i]]
                    elif '_last_two' in attempted[i]:
                        data[f'{thing}_opponent_last_two_proportion'] = data[landed[i]] / data[attempted[i]]
                    elif '_total' in attempted[i]:
                        data[f'{thing}_opponent_total_proportion'] = data[landed[i]] / data[attempted[i]]
                    else:
                        data[f'{thing}_opponent_last_proportion'] = data[landed[i]] / data[attempted[i]]
                else:
                    if '_last_three' in attempted[i]:
                        data[f'{thing}_last_three_proportion'] = data[landed[i]] / data[attempted[i]]
                    elif '_last_two' in attempted[i]:
                        data[f'{thing}_last_two_proportion'] = data[landed[i]] / data[attempted[i]]
                    elif '_total' in attempted[i]:
                        data[f'{thing}_total_proportion'] = data[landed[i]] / data[attempted[i]]
                    else:
                        data[f'{thing}_last_proportion'] = data[landed[i]] / data[attempted[i]]
            else:
                continue

    
    #stats that will get additional rate, eg strike/min
    stats1 = ['KD_1', 'SUB.ATT_1', 'REV._1', 'sig_strike_landed_1', 'sig_strike_attempted_1', 'total_strike_landed_1', 'total_strike_attempted_1', 'take_down_landed_1', 'take_down_attempted_1', 'head_strike_landed_1', 'head_strike_attempted_1', 'body_strike_landed_1', 'body_strike_attempted_1', 'leg_strike_landed_1', 'leg_strike_attempted_1', 'distance_strike_landed_1', 'distance_strike_attempted_1', 'clinch_strike_landed_1', 'clinch_strike_attempted_1', 'ground_strike_landed_1', 'ground_strike_attempted_1', 'total_ctrl_s_1', 'TIME', 'KD_by_opponent', 'SUB.ATT_by_opponent', 'REV._by_opponent', 'sig_strike_landed_by_opponent', 'sig_strike_attempted_by_opponent', 'total_strike_landed_by_opponent', 'total_strike_attempted_by_opponent', 'take_down_landed_by_opponent', 'take_down_attempted_by_opponent', 'head_strike_landed_by_opponent', 'head_strike_attempted_by_opponent', 'body_strike_landed_by_opponent', 'body_strike_attempted_by_opponent', 'leg_strike_landed_by_opponent', 'leg_strike_attempted_by_opponent', 'distance_strike_landed_by_opponent', 'distance_strike_attempted_by_opponent', 'clinch_strike_landed_by_opponent', 'clinch_strike_attempted_by_opponent', 'ground_strike_landed_by_opponent', 'ground_strike_attempted_by_opponent', 'total_ctrl_s_by_opponent', 'KD_1_last_two', 'SUB.ATT_1_last_two', 'REV._1_last_two', 'sig_strike_landed_1_last_two', 'sig_strike_attempted_1_last_two', 'total_strike_landed_1_last_two', 'total_strike_attempted_1_last_two', 'take_down_landed_1_last_two', 'take_down_attempted_1_last_two', 'head_strike_landed_1_last_two', 'head_strike_attempted_1_last_two', 'body_strike_landed_1_last_two', 'body_strike_attempted_1_last_two', 'leg_strike_landed_1_last_two', 'leg_strike_attempted_1_last_two', 'distance_strike_landed_1_last_two', 'distance_strike_attempted_1_last_two', 'clinch_strike_landed_1_last_two', 'clinch_strike_attempted_1_last_two', 'ground_strike_landed_1_last_two', 'ground_strike_attempted_1_last_two', 'total_ctrl_s_1_last_two', 'TIME_last_two', 'KD_by_opponent_last_two', 'SUB.ATT_by_opponent_last_two', 'REV._by_opponent_last_two', 'sig_strike_landed_by_opponent_last_two', 'sig_strike_attempted_by_opponent_last_two', 'total_strike_landed_by_opponent_last_two', 'total_strike_attempted_by_opponent_last_two', 'take_down_landed_by_opponent_last_two', 'take_down_attempted_by_opponent_last_two', 'head_strike_landed_by_opponent_last_two', 'head_strike_attempted_by_opponent_last_two', 'body_strike_landed_by_opponent_last_two', 'body_strike_attempted_by_opponent_last_two', 'leg_strike_landed_by_opponent_last_two', 'leg_strike_attempted_by_opponent_last_two', 'distance_strike_landed_by_opponent_last_two', 'distance_strike_attempted_by_opponent_last_two', 'clinch_strike_landed_by_opponent_last_two', 'clinch_strike_attempted_by_opponent_last_two', 'ground_strike_landed_by_opponent_last_two', 'ground_strike_attempted_by_opponent_last_two', 'total_ctrl_s_by_opponent_last_two', 'KD_1_last_three', 'SUB.ATT_1_last_three', 'REV._1_last_three', 'sig_strike_landed_1_last_three', 'sig_strike_attempted_1_last_three', 'total_strike_landed_1_last_three', 'total_strike_attempted_1_last_three', 'take_down_landed_1_last_three', 'take_down_attempted_1_last_three', 'head_strike_landed_1_last_three', 'head_strike_attempted_1_last_three', 'body_strike_landed_1_last_three', 'body_strike_attempted_1_last_three', 'leg_strike_landed_1_last_three', 'leg_strike_attempted_1_last_three', 'distance_strike_landed_1_last_three', 'distance_strike_attempted_1_last_three', 'clinch_strike_landed_1_last_three', 'clinch_strike_attempted_1_last_three', 'ground_strike_landed_1_last_three', 'ground_strike_attempted_1_last_three', 'total_ctrl_s_1_last_three', 'TIME_last_three', 'KD_by_opponent_last_three', 'SUB.ATT_by_opponent_last_three', 'REV._by_opponent_last_three', 'sig_strike_landed_by_opponent_last_three', 'sig_strike_attempted_by_opponent_last_three', 'total_strike_landed_by_opponent_last_three', 'total_strike_attempted_by_opponent_last_three', 'take_down_landed_by_opponent_last_three', 'take_down_attempted_by_opponent_last_three', 'head_strike_landed_by_opponent_last_three', 'head_strike_attempted_by_opponent_last_three', 'body_strike_landed_by_opponent_last_three', 'body_strike_attempted_by_opponent_last_three', 'leg_strike_landed_by_opponent_last_three', 'leg_strike_attempted_by_opponent_last_three', 'distance_strike_landed_by_opponent_last_three', 'distance_strike_attempted_by_opponent_last_three', 'clinch_strike_landed_by_opponent_last_three', 'clinch_strike_attempted_by_opponent_last_three', 'ground_strike_landed_by_opponent_last_three', 'ground_strike_attempted_by_opponent_last_three', 'total_ctrl_s_by_opponent_last_three', 'KD_1_total', 'SUB.ATT_1_total', 'REV._1_total', 'sig_strike_landed_1_total', 'sig_strike_attempted_1_total', 'total_strike_landed_1_total', 'total_strike_attempted_1_total', 'take_down_landed_1_total', 'take_down_attempted_1_total', 'head_strike_landed_1_total', 'head_strike_attempted_1_total', 'body_strike_landed_1_total', 'body_strike_attempted_1_total', 'leg_strike_landed_1_total', 'leg_strike_attempted_1_total', 'distance_strike_landed_1_total', 'distance_strike_attempted_1_total', 'clinch_strike_landed_1_total', 'clinch_strike_attempted_1_total', 'ground_strike_landed_1_total', 'ground_strike_attempted_1_total', 'total_ctrl_s_1_total', 'TIME_total', 'KD_by_opponent_total', 'SUB.ATT_by_opponent_total', 'REV._by_opponent_total', 'sig_strike_landed_by_opponent_total', 'sig_strike_attempted_by_opponent_total', 'total_strike_landed_by_opponent_total', 'total_strike_attempted_by_opponent_total', 'take_down_landed_by_opponent_total', 'take_down_attempted_by_opponent_total', 'head_strike_landed_by_opponent_total', 'head_strike_attempted_by_opponent_total', 'body_strike_landed_by_opponent_total', 'body_strike_attempted_by_opponent_total', 'leg_strike_landed_by_opponent_total', 'leg_strike_attempted_by_opponent_total', 'distance_strike_landed_by_opponent_total', 'distance_strike_attempted_by_opponent_total', 'clinch_strike_landed_by_opponent_total', 'clinch_strike_attempted_by_opponent_total', 'ground_strike_landed_by_opponent_total', 'ground_strike_attempted_by_opponent_total', 'total_ctrl_s_by_opponent_total']
    
    for thing in stats1:
        if '_total' in thing:
            data[f'{thing}_per_min'] = data[thing] / (data['TIME_total']/60)
        elif '_last_two' in thing:
            data[f'{thing}_per_min'] = data[thing] / (data['TIME_last_two']/60)
        elif '_last_three' in thing:
            data[f'{thing}_per_min'] = data[thing] / (data['TIME_last_three']/60)
        else:
            data[f'{thing}_per_min'] = data[thing] / (data['TIME']/60)
            
    for cols in data.columns:
        if 'index' in cols:
            data.pop(cols)
        else:
            continue
    
    #categorical data to numeric
    categorical_to_numeric = ['FIGHTER', 'FIGHTER2', 'STANCE_1', 'STANCE_opponent', 'TIME FORMAT']
    for thing in categorical_to_numeric:
        data[f'{thing.lower()}_codes'] = data[thing].astype('category').cat.codes
    
    #get current stats for predictions
    current = current_stats(data)
    
    data = data.drop(columns=[])
    return data, current

new_stats = fight_stats('/Users/joelempster/Downloads/ufc_fight_stats (7).csv',
                        '/Users/joelempster/Downloads/ufc_event_details (8).csv',
                        '/Users/joelempster/Downloads/ufc_fight_results (7).csv',
                        '/Users/joelempster/Downloads/ufc_fighter_tott (7).csv')
new_stats[0]

42 columns were dropped


/var/folders/0r/n_w4gf9j7y9bgvylwd22h9dc0000gn/T/ipykernel_83174/1697731942.py:92: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[rolling_last_three] = data.groupby('FIGHTER')[columns_to_accumulate].rolling(window=3).sum().reset_index(level=0, drop=True)
/var/folders/0r/n_w4gf9j7y9bgvylwd22h9dc0000gn/T/ipykernel_83174/1697731942.py:92: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[rolling_last_three] = data.groupby('FIGHTER')[columns_to_accumulate].rolling(window=3).sum().reset_index(level=0, drop=True)
/var/folders/0r/

,EVENT,BOUT,FIGHTER,KD_1,SUB.ATT_1,REV._1,sig_strike_landed_1,sig_strike_attempted_1,total_strike_landed_1,total_strike_attempted_1,...,clinch_strike_landed_by_opponent_total_per_min,clinch_strike_attempted_by_opponent_total_per_min,ground_strike_landed_by_opponent_total_per_min,ground_strike_attempted_by_opponent_total_per_min,total_ctrl_s_by_opponent_total_per_min,fighter_codes,fighter2_codes,stance_1_codes,stance_opponent_codes,time format_codes
0,UFC Fight Night: Rozenstruik vs. Gaziev,Ludovit Klein vs. AJ Cunningham,AJ Cunningham,0,0,0,36,75,36,77,...,0.434783,0.434783,0.869565,1.086957,5.869565,0,1500,-1,2,15
1,UFC 271: Adesanya vs. Whittaker 2,AJ Dobson vs. Jacob Malkoun,AJ Dobson,0,0,0,52,90,68,110,...,0.266667,0.466667,3.133333,4.266667,37.133333,1,995,1,1,15
2,UFC 280: Oliveira vs. Makhachev,Armen Petrosyan vs. AJ Dobson,AJ Dobson,0,0,0,58,161,64,168,...,0.533333,0.666667,1.566667,2.133333,18.566667,1,189,1,1,15
3,UFC Fight Night: Luque vs. Dos Anjos,AJ Dobson vs. Tafon Nchukwi,AJ Dobson,0,0,0,67,134,98,176,...,0.444444,0.600000,1.044444,1.422222,12.377778,1,2267,1,1,15
4,UFC Fight Night: Ribas vs. Namajunas,Edmen Shahbazyan vs. AJ Dobson,AJ Dobson,0,0,1,15,46,20,54,...,0.423814,0.605449,1.150353,1.614531,12.230071,1,722,1,1,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15623,UFC 253: Adesanya vs. Costa,Hakeem Dawodu vs. Zubaira Tukhugov,Zubaira Tukhugov,0,0,0,35,93,40,98,...,0.084643,0.193470,0.060459,0.060459,7.194680,2526,898,1,1,15
15624,UFC 267: Blachowicz vs. Teixeira,Ricardo Ramos vs. Zubaira Tukhugov,Zubaira Tukhugov,0,0,0,77,181,78,185,...,0.081883,0.174002,0.051177,0.051177,6.090072,2526,1987,1,1,15
15625,UFC 284: Makhachev vs. Volkanovski,Zubaira Tukhugov vs. Elves Brener,Zubaira Tukhugov,0,0,0,70,169,73,172,...,0.097604,0.186335,0.044366,0.044366,5.279503,2526,739,1,-1,15
15626,UFC Fight Night: Costa vs. Vettori,Jonathan Martinez vs. Zviad Lazishvili,Zviad Lazishvili,0,0,0,63,160,66,163,...,0.000000,0.000000,0.000000,0.000000,0.000000,2527,1225,1,2,15


In [39]:
conor_vs_chandler = compare_fighters('Conor McGregor', 'Michael Chandler')

,FIGHTER,KD_1,SUB.ATT_1,REV._1,sig_strike_landed_1,sig_strike_attempted_1,total_strike_landed_1,total_strike_attempted_1,take_down_landed_1,take_down_attempted_1,...,clinch_strike_landed_by_opponent_total_per_min,clinch_strike_attempted_by_opponent_total_per_min,ground_strike_landed_by_opponent_total_per_min,ground_strike_attempted_by_opponent_total_per_min,total_ctrl_s_by_opponent_total_per_min,fighter_codes,fighter2_codes,stance_1_codes,stance_opponent_codes,time format_codes
3044,Conor McGregor,0,1,0,27,38,43,54,0,0,...,0.852954,1.261661,0.861839,1.270546,16.472679,493,703,2,2,17
10488,Michael Chandler,0,0,0,43,100,65,135,3,7,...,0.274314,0.299252,0.573566,0.922693,6.134663,1675,703,1,2,15
